In [2]:
# Import packages
import pandas as pd
import numpy as np
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split

In [3]:
# Load dataset
# Import Dataset 
X_train = pd.read_csv('../data/processed/X_train.csv')
X_val = pd.read_csv('../data/processed/X_val.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')
y_train = pd.read_csv('../data/processed/y_train.csv')
y_val = pd.read_csv('../data/processed/y_val.csv')
y_test = pd.read_csv('../data/processed/y_test.csv')


In [4]:
from joblib import load

In [5]:
# Load preprocessor
preprocessor= load('../models/preprocessor/preprocessor.pkl')

In [13]:

from sklearn.ensemble import ExtraTreesRegressor

# Create a pipeline with the preprocessor and the ExtraTreesRegressor model
extratree_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', ExtraTreesRegressor(
        n_estimators=200,
        max_depth=20,               # Increase to allow more depth
        min_samples_split=3,        # Reduce to allow splits with fewer samples
        min_samples_leaf=2,         # Reduce for more flexible leaf nodes
        max_features=0.8,           # Increase max_features for potentially better splits
        random_state=42
    ))
])

In [14]:
from sklearn.utils import resample

# Randomly sample 20% of the training data
X_train_subset, y_train_subset = resample(X_train, y_train, n_samples=int(0.2 * len(X_train)), random_state=42)

# Fit the pipeline on the subset of the data
extratree_pipeline.fit(X_train_subset, y_train_subset)

C:\Users\sovan\AppData\Local\pypoetry\Cache\virtualenvs\adv-mla-at3-oSBB06he-py3.12\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\sovan\AppData\Local\pypoetry\Cache\virtualenvs\adv-mla-at3-oSBB06he-py3.12\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cabins',
                                                  OrdinalEncoder(categories=[['no_stop',
                                                                              'coach',
                                                                              'premium '
                                                                              'coach',
                                                                              'business',
                                                                              'first'],
                                                                             ['no_stop',
                                                                              'coach',
                                                                              'premium '
                                                                              'coach',
                                                                              'business',
                                                                              'first'],
                                                                             ['no_stop',
                                                                              'coach',
                                                                              'premium '
                                                                              'coach',
                                                                              'business',
                                                                              'first'],
                                                                             ['no_stop',
                                                                              'coach',
                                                                              'premium '
                                                                              'coach',
                                                                              'business',
                                                                              'first']])...
                                                   'Cabin_Leg3',
                                                   'Cabin_Leg4']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['startingAirport',
                                                   'destinationAirport']),
                                                 ('standard', StandardScaler(),
                                                  ['date_diff', 'month', 'day',
                                                   'hour', 'minute',
                                                   'day_of_week',
                                                   'week_of_year',
                                                   'medianTravelDistance'])])),
                ('model',
                 ExtraTreesRegressor(max_depth=20, max_features=0.8,
                                     min_samples_leaf=2, min_samples_split=3,
                                     n_estimators=200, random_state=42))])

In [20]:
preds_train = extratree_pipeline.predict(X_train)
preds_val = extratree_pipeline.predict(X_val)
preds_test = extratree_pipeline.predict(X_test)

In [21]:
from my_krml_ratana.models.performance import print_regressor_scores

In [22]:
print_regressor_scores(y_preds=preds_train, y_actuals=y_train, set_name='Training')

RMSE Training: 85.90165090208784
MAE Training: 55.9700324556342


In [23]:
print_regressor_scores(y_preds=preds_val, y_actuals=y_val, set_name='Validating')

RMSE Validating: 87.17596744494192
MAE Validating: 56.741541960951515


In [24]:
print_regressor_scores(y_preds=preds_test, y_actuals=y_test, set_name='Testing')

RMSE Testing: 87.60671993842296
MAE Testing: 56.78772546973607
